In [265]:
import numpy as np 
import json
import pandas as pd
import geocoder
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

In [266]:
CLIENT_ID = 'IJ5XCHFSSXUAUIYUYYSOUCVSGLZK241BNGEEGT252CIY14N5' # your Foursquare ID
CLIENT_SECRET = 'CRXPEJBKAAKI1YLZGSRO0QWOC22AVGJ3SG1OGENV2PBJGUNP' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
radius = 500

def get_latlng(neigh):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Virginia Beach, Virginia, United States'.format(neigh))
        lat_lng_coords = g.latlng
    return lat_lng_coords




In [267]:
vbneigh = {'Neighborhoods': ['Alanton', 'Aragona Village', 'Bay Colony', 'Bayside', 'Cape Henry', 'Chesapeake Beach', 'Croatan Beach','Great Neck Point', 'Green Run', 'Kempsville', 'Lago Mar', 'Larkspur', 'London Bridge', 'Lynnhaven', 'Newtown', 'The North End', 'Oceana', 'Ocean Park', 'Pembroke Manor', 'Princess Anne', 'Pungo', 'Red Mill Commons', 'Sandbridge', 'Thalia', 'Thoroughgood']}
vbneighdf= pd.DataFrame(vbneigh)
coordsvb = [ get_latlng(neigh) for neigh in vbneighdf['Neighborhoods']]
dfvbc = pd.DataFrame(coordsvb, columns=['Latitude', 'Longitude'])
vbneighdf['Latitude'] = dfvbc['Latitude']
vbneighdf['Longitude'] = dfvbc['Longitude']
print(vbneighdf.shape)
vbneighdf.head()

(25, 3)


,Neighborhoods,Latitude,Longitude
0,Alanton,36.882650,-76.021880
1,Aragona Village,36.848770,-76.149380
2,Bay Colony,36.878110,-76.006830
3,Bayside,36.902120,-76.133070
4,Cape Henry,36.922588,-76.000312


In [268]:
address = 'Virginia Beach, Virginia, United States'
geolocator = Nominatim(user_agent="foursquare-agent")
location = geolocator.geocode(address)
latitudevb = location.latitude
longitudevb = location.longitude
print('The coordinated of VB {}, {}.'.format(latitudevb, longitudevb))
mapvb = folium.Map(location=[latitudevb, longitudevb], zoom_start=11)
for lat, lng, neighborhood in zip(vbneighdf['Latitude'],  vbneighdf['Longitude'], vbneighdf['Neighborhoods']):
 label = '{}'.format(neighborhood)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=5,popup=label,color='#93b6ed',fill=True,fill_color='#c0d7fa',fill_opacity=0.7).add_to(mapvb)
mapvb

The coordinated of VB 36.8529841, -75.9774183.


In [269]:
om={'Name':['Aksarben/Elmwood Park'
,'Aksarben Village',
'Applewood',
'Baker Place',
'Bemis Park',
'Bennington',
'Benson',
'Bent Creek',
'Blackstone',
'Boys Town',
'Briar Hills',
'Bridlewood',
'Brookhaven',
'Buena Vista',
'Burlington Road',
'Burnt District',
'Cathedral Landmark District',
'Carter Lake',
'Chalco',
'Conestoga Place',
'Country Club',
'Dahlman',
'Deer Park',
'Dundee',
'Eagle Ridge',
'Elkhorn',
'Elmwood Park',
'Fairacres',
'Florence',
'Field Club',
'First National Business Park',
'Fountain Hills',
'Gifford Park',
'Gold Coast',
'Hanscom Park',
'Happy Hollow',
'Hillsborough',
'Howard Street Apartment District',
'Huntington Park',
'Irvington',
'Keystone',
'Kountze Place',
'La Vista',
'Leavenworth',
'Little Bohemia',
'Little Italy',
'Maple Village',
'Meridian Park',
'Midtown Crossing',
'Millard',
'Miller Park',
'Minne Lusa',
'Morton Meadows',
'Mockingbird Heights',
'Near North Side',
'Oak Heights',
'Old Gold Coast',
'Old Market Historic District',
'Omaha View',
'Orchard Hill',
'Original Montclair (Omaha)',
'Papillion',
'Park Avenue',
'Park East',
'Pepperwood',
'Prospect Hill',
'Ralston',
'Raven Oaks',
'Roanoke',
'Robin Hill',
'Royalwood Estates',
'Regency',
'Roxbury',
'Saratoga',
'Sheelytown',
'South Omaha Main Street Historic District',
'Sporting District',
'Spring Lake',
"Squatter's Row",
'Sunny Slope',
'Trendwood',
'Valley',
'Vinton Street Commercial Historic District','Walnut Hill','Waterloo','Westgate','Westside']}
omneigh=pd.DataFrame(om)
omneigh.dropna(inplace=True)
omneigh.head(88)

,Name
0,Aksarben/Elmwood Park
1,Aksarben Village
2,Applewood
3,Baker Place
4,Bemis Park
...,...
82,Vinton Street Commercial Historic District
83,Walnut Hill
84,Waterloo
85,Westgate


In [270]:
def get_latlng(neigh):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Omaha, Nebraska, United States'.format(neigh))
        lat_lng_coords = g.latlng
    return lat_lng_coords
coordsom = [get_latlng(neigh) for neigh in omneigh['Name']]
dfomc = pd.DataFrame(coordsom, columns=['Latitude', 'Longitude'])
omneigh['Latitude'] = dfomc['Latitude']
omneigh['Longitude'] = dfomc['Longitude']


In [271]:
omneigh

,Name,Latitude,Longitude
0,Aksarben/Elmwood Park,41.244640,-96.001850
1,Aksarben Village,41.238940,-96.013050
2,Applewood,41.291367,-96.059183
3,Baker Place,41.329601,-96.079162
4,Bemis Park,41.269970,-95.968270
...,...,...,...
82,Vinton Street Commercial Historic District,41.229189,-96.206194
83,Walnut Hill,41.270490,-95.976100
84,Waterloo,41.287950,-96.289090
85,Westgate,41.226310,-96.032685


In [272]:
geolocator = Nominatim(user_agent="foursquare-agent")
location = geolocator.geocode(address)
latitudeom = location.latitude
longitudeom = location.longitude
print('The coordinated of Omaha {}, {}.'.format(latitudeom, longitudeom))
mapom = folium.Map(location=[latitudeom, longitudeom], zoom_start=11)
for lat, lng, neighborhood in zip(omneigh['Latitude'], omneigh['Longitude'],omneigh['Name']):
 label = '{}'.format(neighborhood)
 label = folium.Popup(label, parse_html=True)
 folium.CircleMarker([lat, lng],radius=5,popup=label,color='#f5c951',fill=True,fill_color='#edd28a',fill_opacity=0.7).add_to(mapom)
mapom

The coordinated of Omaha 36.8529841, -75.9774183.


In [273]:
neighlatvb = vbneighdf.loc[0, 'Latitude'] 
neighlongvb = vbneighdf.loc[0, 'Longitude'] 
neighnamevb = vbneighdf.loc[0, 'Neighborhoods']
neighlatom = omneigh.loc[0, 'Latitude'] 
neighlongom = omneigh.loc[0, 'Longitude'] 
neighnameom = omneigh.loc[0, 'Name']

In [274]:
urlvb ='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighlatvb, neighlongvb, VERSION, radius, LIMIT)
urlom ='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighlatom, neighlongom, VERSION, radius, LIMIT)
resultsvb = requests.get(urlvb).json()
resultsvb

{'meta': {'code': 200, 'requestId': '5fb4496132abda5e08429663'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Alanton',
  'headerFullLocation': 'Alanton, Virginia Beach',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 36.88715000450007,
    'lng': -76.01626455939116},
   'sw': {'lat': 36.87814999550007, 'lng': -76.02749544060875}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57b51265498ed849ffa879a3',
       'name': 'Apeshyt Hambone',
       'location': {'address': 'Great Neck',
        'lat': 36.884174,
        'lng': -76.022644,
        'labeledLatLngs': [{'label': 'display',
      

In [275]:
resultsom = requests.get(urlom).json()
resultsom

{'meta': {'code': 200, 'requestId': '5fb44962d859ad1d9f258335'},
 'response': {'headerLocation': 'Aksarben - Elmwood Park',
  'headerFullLocation': 'Aksarben - Elmwood Park, Omaha',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 41.24914000450006,
    'lng': -95.99587634179906},
   'sw': {'lat': 41.24013999550006, 'lng': -96.00782365820092}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '52e5be5d11d2e1f4a93adf07',
       'name': 'Tracks Lounge',
       'location': {'address': '1506 S 60th St',
        'lat': 41.24470837404333,
        'lng': -96.00492908044346,
        'labeledLatLngs': [{'label': 'display',
          'lat': 41.24470837404333,
          'lng': -96.00492908044346},
         {'label': 'entrance', '

In [276]:
def getcata(row):
    try:
        categorlist = row['categories']
    except:
        categorlist = row['venue.categories']
        
    if len(categorlist) == 0:
        return None
    else:
        return categorlist[0]['name']

In [277]:
venuesvb = resultsvb['response']['groups'][0]['items']
venuesom = resultsom['response']['groups'][0]['items']
nearvenuesvb = json_normalize(venuesvb)
nearvenuesom = json_normalize(venuesom)
filtcolumns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearvenuesvb =nearvenuesvb.loc[:, filtcolumns]
nearvenuesvb['venue.categories'] = nearvenuesvb.apply(getcata, axis=1)
nearvenuesvb.columns = [col.split(".")[-1] for col in nearvenuesvb.columns]
nearvenuesvb.head()
nearvenuesom =nearvenuesom.loc[:, filtcolumns]
nearvenuesom['venue.categories'] = nearvenuesom.apply(getcata, axis=1)
nearvenuesom.columns = [col.split(".")[-1] for col in nearvenuesom.columns]
nearvenuesom.head(90)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Tracks Lounge,Bar,41.244708,-96.004929
1,Jonesy's Taco House,Taco Place,41.244855,-96.005167
2,Boyd & Charlies BBQ,BBQ Joint,41.245421,-96.005196
3,The Gentelman's Room,Historic Site,41.242223,-96.000026
4,Take 1 Talent,Concert Hall,41.248665,-96.003366


In [278]:
nearvenuesvb.head()

,name,categories,lat,lng
0,Apeshyt Hambone,Comedy Club,36.884174,-76.022644
